In [ ]:
import pandas as pd
from test_definition import load_and_transform, create_demand
import pyomo.environ as pyo
from multiprocessing import Pool

# Data import

In [ ]:
nodes, channels = load_and_transform("../data/network_graph_2024_06_12.json")
nodes = create_demand(nodes)

In [ ]:
## In order to accomplish that, the nodes dataset is split into some chuncks,
## then the execution of the finding function is parallelized for every
## dataframe chunck.
## TODO: Process executed before creation of directed edges in the channels dataframe

def nodes_df_splitting(pd_object: pd.DataFrame, n: int) -> list:
    results = []
    splitting: int = len(pd_object) // n
    for i in range(n):
        ris = pd_object[i * splitting : (i+1) * splitting]
        results.append(ris)
    if len(pd_object) % n != 0:
        ris = pd_object[n * splitting: len(pd_object)]
        results.append(ris)
    return results


def find_channels(n: str) -> list:
    """
    :param n: node pub key
    :return: list of channels for the node
    Note that the listed channels are directed channels
    that have also a mirrored channel that describes the
    flow of funds in the opposite direction.
    Thus, this channels list is simply adapted to other
    flow of channels by considering then the channels
    with id "INV-<channel_id>"
    """
    channels_list = []
    for c in channels.index:
        if channels.loc[c, "node1_pub"] == n.name:
            channels_list.append(c)
    return channels_list


def parallel_channel_finding(args: tuple) -> pd.DataFrame:
    dfs, i = args
    df = dfs[i].copy()
    df["outgoing_channels"] = df.apply(find_channels, axis=1)
    return df


def append_inv_channel(c: list) -> list:
    ris = []
    for i in c:
        ris.append("INV" + str(i))
    return ris


def flipped_channels(pd_object: pd.DataFrame) -> pd.DataFrame:
    pd_object["incoming_channels"] = pd_object["outgoing_channels"].apply(append_inv_channel)
    return pd_object

In [ ]:
slices = 10
dataframes = nodes_df_splitting(nodes, slices)

pool = Pool() # to set custom number of processes use processes=n
inputs: list = [(dataframes, y) for y in range(slices)]
outputs: list = pool.map(parallel_channel_finding, inputs)


In [ ]:
nodes = pd.concat(outputs)
nodes = flipped_channels(nodes)

nodes.to_pickle("../data/nodes.pkl")

## Modeling

In [ ]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=channels.index) #within=model.NODES*model.NODES)

In [ ]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

In [ ]:
def objective_function(model: pyo.ConcreteModel):
    return sum(channels.loc[k, "base_fee"] * model.x[k] for k in model.CHANNELS) + sum(channels.loc[i, "rate_fee"] * model.a[i] for i in model.CHANNELS)

model.totalCost = pyo.Objective(expr=objective_function(model), sense=pyo.minimize)

### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$


In [ ]:
def capacity_constraint(model: pyo.ConcreteModel, a):
    return model.a[a] <= channels.loc[a, "capacity"] * model.x[a]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [ ]:
#def compute_outgoing(n: str) -> list:
#    """
#    Compute outgoing channels list for the node n
#    :param n: node identifier
#    :return: list of outgoing channels for node n
#    """
#    return [c for c in model.CHANNELS if channels.loc[c, "node1_pub"] == n]
#
#
#def compute_incoming(n: str) -> list:
#    """
#    Compute incoming channels list for the node n
#    :param n: node identifier
#    :return: list of incoming channels for node n
#    """
#    return [c for c in model.CHANNELS if channels.loc[c, "node2_pub"] == n]


#def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
#    print("Start processing")
#    return sum(model.a[a] for a in compute_incoming(f"{n}")) - sum(model.a[a] for a in compute_outgoing(f"{n}")) == nodes.loc[n, "demand"]


def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    print("Start processing")
    return sum(model.a[a] for a in nodes.loc[n, 'incoming_channels']) - sum(model.a[a] for a in nodes.loc[n, 'outgoing_channels']) == nodes.loc[n, "demand"]

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

## Solving the model

In [ ]:
opt = pyo.SolverFactory('cbc')
opt.solve(model, tee=True)

In [ ]:
model.x.pprint()